In [ ]:

!nvidia-smi

In [ ]:
import os
import glob
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from segment_anything import sam_model_registry
from segment_anything.utils.transforms import ResizeLongestSide
import matplotlib.pyplot as plt
import random
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Subset

In [ ]:
def seed_everything(seed: int = 42) -> None:
    os.environ["PYTHONHASHSEED"] = str(seed)           
    random.seed(seed)                                 
    np.random.seed(seed)                              
    torch.manual_seed(seed)                           
    torch.cuda.manual_seed(seed)                       
    torch.cuda.manual_seed_all(seed)                   
    torch.use_deterministic_algorithms(True, warn_only=True)
    torch.backends.cudnn.deterministic = True         
    torch.backends.cudnn.benchmark = False 

In [ ]:

# Configuration


dataset_path = "./final_dataset"               # root containing train/ & test/
checkpoint_path = "./sam_vit_h_4b8939.pth"     # SAM ViT‑H checkpoint
model_type = "vit_h"                           # sam encoder type
output_dir = "./predictions"                   # where to save models & figs
os.makedirs(output_dir, exist_ok=True)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:

# Dataset


class RetinalHemorrhageDataset(Dataset):
    """Dataset loading RGB fundus images and binary masks."""

    def __init__(self, root_dir: str, split: str = "train", transform=None):
        self.root_dir = root_dir
        self.split = split
        self.transform = transform
        self.images_dir = os.path.join(root_dir, split, "images")
        self.masks_dir = os.path.join(root_dir, split, "masks")
        self.image_files = sorted(glob.glob(os.path.join(self.images_dir, "*.*")))
        self.mask_files = sorted(glob.glob(os.path.join(self.masks_dir, "*.*")))

        assert len(self.image_files) == len(self.mask_files), (
            f"Number of images ({len(self.image_files)}) and masks "
            f"({len(self.mask_files)}) don't match!"
        )
        print(f"Found {len(self.image_files)} samples in '{split}' split")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Image
        img_path = self.image_files[idx]
        img = cv2.imread(img_path)
        if img is None:
            raise ValueError(f"Failed to load image: {img_path}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Mask
        msk_path = self.mask_files[idx]
        msk = cv2.imread(msk_path, cv2.IMREAD_GRAYSCALE)
        if msk is None:
            raise ValueError(f"Failed to load mask: {msk_path}")
        msk = (msk > 0).astype(np.float32)

        # To tensor
        img = torch.from_numpy(img).permute(2, 0, 1).float() / 255.0  # C,H,W
        msk = torch.from_numpy(msk).unsqueeze(0).float()              # 1,H,W
        return {"image": img, "mask": msk, "filename": os.path.basename(img_path)}


# Loss Function



class FocalTverskyLoss(nn.Module):
    def __init__(self, alpha=0.3, beta=0.7, gamma=0.75, eps=1e-6):
        super(FocalTverskyLoss, self).__init__()
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.eps = eps

    def forward(self, inputs, targets):
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        TP = (inputs * targets).sum()
        FP = ((1 - targets) * inputs).sum()
        FN = (targets * (1 - inputs)).sum()

        tversky = (TP + self.eps) / (TP + self.alpha * FP + self.beta * FN + self.eps)
        focal_tversky = (1 - tversky) ** self.gamma

        return focal_tversky


In [ ]:

# U‑Net style decoder blocks


class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.net(x)

class UpBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.upsample = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=False)
        self.conv = DoubleConv(in_ch, out_ch)

    def forward(self, x):
        x = self.upsample(x)
        return self.conv(x)

class UNetDecoder(nn.Module):

    def __init__(self, in_ch: int = 256):
        super().__init__()
        self.block1 = DoubleConv(in_ch, 256)
        self.up1 = UpBlock(256, 128)
        self.up2 = UpBlock(128, 64)
        self.up3 = UpBlock(64, 32)
        self.up4 = UpBlock(32, 16)
        self.final = nn.Conv2d(16, 1, kernel_size=1)

    def forward(self, x):
        x = self.block1(x)
        x = self.up1(x)
        x = self.up2(x)
        x = self.up3(x)
        x = self.up4(x)
        return self.final(x)


# Fine‑tuner combining SAM encoder + U‑Net decoder

class SAMFineTuner(nn.Module):
    def __init__(self, checkpoint_path: str, model_type: str):
        super().__init__()
        print(f"Loading SAM encoder '{model_type}' …")
        self.sam = sam_model_registry[model_type](checkpoint=checkpoint_path)
        self.sam.to(device)
        print("SAM encoder loaded.")

        # Freeze encoder parameters
        for p in self.sam.image_encoder.parameters():
            p.requires_grad = False

        # Decoder
        self.decoder = UNetDecoder(in_ch=256)

        # Pre‑processing helper
        self.transform = ResizeLongestSide(self.sam.image_encoder.img_size)
        self.pixel_mean = torch.tensor([123.675, 116.28, 103.53]).view(3, 1, 1)
        self.pixel_std = torch.tensor([58.395, 57.12, 57.375]).view(3, 1, 1)

    @torch.no_grad()
    def preprocess_single(self, img_tensor: torch.Tensor) -> torch.Tensor:
       
        img_np = (img_tensor.cpu().numpy().transpose(1, 2, 0) * 255.0).astype(np.float32)
        img_np = self.transform.apply_image(img_np)
        t = torch.from_numpy(img_np).permute(2, 0, 1).float()
        t = (t - self.pixel_mean) / self.pixel_std
        return t

    def forward(self, imgs: torch.Tensor):  # imgs: B,C,H,W in [0,1]
        B, _, H, W = imgs.shape
        processed = torch.stack([self.preprocess_single(im) for im in imgs]).to(device)

        with torch.no_grad():
            emb = self.sam.image_encoder(processed)  # B,256,h,w

        dec_out = self.decoder(emb)                 # B,1,h',w'
        dec_out = F.interpolate(dec_out, size=(H, W), mode="bilinear", align_corners=False)
        return torch.sigmoid(dec_out)

In [ ]:

# Metrics


def compute_metrics(y_true: np.ndarray, y_pred: np.ndarray, thr: float = 0.5):
    y_pred_bin = (y_pred > thr).astype(np.uint8)
    y_true = y_true.astype(np.uint8)
    tp = np.sum((y_pred_bin == 1) & (y_true == 1))
    fp = np.sum((y_pred_bin == 1) & (y_true == 0))
    fn = np.sum((y_pred_bin == 0) & (y_true == 1))
    tn = np.sum((y_pred_bin == 0) & (y_true == 0))

    acc = (tp + tn) / (tp + fp + fn + tn + 1e-6)
    precision = tp / (tp + fp + 1e-6)
    recall = tp / (tp + fn + 1e-6)
    f1 = 2 * precision * recall / (precision + recall + 1e-6)
    iou = tp / (tp + fp + fn + 1e-6)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1_score": f1, "jaccard": iou}


# Train / Eval loops


def train_epoch(model, loader, criterion, optimizer, epoch):
    model.train()
    total_loss = 0.0
    pbar = tqdm(loader, desc=f"Epoch {epoch}")
    for batch in pbar:
        imgs = batch["image"].to(device)
        msks = batch["mask"].to(device)
        preds = model(imgs)
        loss = criterion(preds, msks)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        pbar.set_postfix(loss=f"{loss.item():.4f}")
    return total_loss / len(loader)

def evaluate(model, loader, criterion, save_preds: bool = False):
    model.eval()
    val_loss = 0.0
    stats = {k: [] for k in ["accuracy", "precision", "recall", "f1_score", "jaccard"]}
    pbar = tqdm(loader, desc="Eval")
    with torch.no_grad():
        for batch in pbar:
            imgs = batch["image"].to(device)
            msks = batch["mask"].to(device)
            fnames = batch["filename"]
            preds = model(imgs)
            loss = criterion(preds, msks)
            val_loss += loss.item()
            preds_np = preds.cpu().numpy()
            msks_np = msks.cpu().numpy()
            for i in range(len(imgs)):
                metrics = compute_metrics(msks_np[i, 0], preds_np[i, 0])
                for k in stats:
                    stats[k].append(metrics[k])
                if save_preds:
                    save_visualization(imgs[i], msks_np[i, 0], preds_np[i, 0], metrics, fnames[i])
    avg_stats = {k: float(np.mean(v)) for k, v in stats.items()}
    return val_loss / len(loader), avg_stats


# Helper to save side‑by‑side visualisations


def save_visualization(img_t, true_msk, pred_msk, metrics, fname):
    img = (img_t.cpu().numpy().transpose(1, 2, 0) * 255).astype(np.uint8)
    pred = (pred_msk > 0.5).astype(np.uint8) * 255
    true = (true_msk > 0.5).astype(np.uint8) * 255

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    axes[0].imshow(img)
    axes[0].set_title("Image")
    axes[0].axis("off")
    axes[1].imshow(true, cmap="gray")
    axes[1].set_title("Ground Truth")
    axes[1].axis("off")
    axes[2].imshow(pred, cmap="gray")
    axes[2].set_title("Prediction")
    axes[2].axis("off")

    txt = (
        f"IoU={metrics['jaccard']:.3f} | F1={metrics['f1_score']:.3f} | "
        f"Prec={metrics['precision']:.3f} | Rec={metrics['recall']:.3f}"
    )
    
    # Add more space at the bottom for the text
    plt.subplots_adjust(bottom=0.2)
    fig.text(0.5, 0.05, txt, ha="center", fontsize=10)

    out_path = os.path.join(output_dir, f"{os.path.splitext(fname)[0]}_viz.png")
    plt.savefig(out_path, dpi=150, bbox_inches='tight')
    plt.close()


In [ ]:
# Seeding
seed_everything(42)

# Data
train_ds = RetinalHemorrhageDataset(dataset_path, split="train")
full_test_ds = RetinalHemorrhageDataset(dataset_path, split="test")

# Split test dataset into fixed 50% validation and 50% test
val_size = len(full_test_ds) // 2
test_size = len(full_test_ds) - val_size
val_ds, test_ds = torch.utils.data.random_split(full_test_ds, [val_size, test_size], generator=torch.Generator().manual_seed(42))

# Dataloaders
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=0, pin_memory=True)
test_loader = DataLoader(test_ds, batch_size=1, shuffle=False, num_workers=0, pin_memory=True)

# Model / loss / optim
model = SAMFineTuner(checkpoint_path, model_type).to(device)
criterion = FocalTverskyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
scheduler = ReduceLROnPlateau(
    optimizer,
    mode="min",
    factor=0.1,
    patience=4,
    threshold=1e-4,
    verbose=True,
)


In [9]:
# Training loop with early stopping
num_epochs = 50
best_f1 = -1.0
patience = 10
no_improve_epochs = 0

for epoch in range(1, num_epochs + 1):
    tr_loss = train_epoch(model, train_loader, criterion, optimizer, epoch)
    val_loss, metrics = evaluate(model, val_loader, criterion)
    scheduler.step(val_loss)
    lr_now = optimizer.param_groups[0]["lr"]

    print(
        f"Epoch {epoch}/{num_epochs} | "
        f"Train Loss: {tr_loss:.4f} | Val Loss: {val_loss:.4f} | "
        f"IoU: {metrics['jaccard']:.4f} | "
        f"F1: {metrics['f1_score']:.4f} | "
        f"Precision: {metrics['precision']:.4f} | "
        f"Recall: {metrics['recall']:.4f} | "
        f"Accuracy: {metrics['accuracy']:.4f}"
    )

    if metrics["f1_score"] > best_f1:
        best_f1 = metrics["f1_score"]
        no_improve_epochs = 0  # reset counter
        torch.save(model.state_dict(), os.path.join(output_dir, "best_model.pth"))
        print(f"New best model saved (F1: {best_f1:.4f})")
    else:
        no_improve_epochs += 1
        print(f"No improvement in F1 for {no_improve_epochs} epoch(s)")

    print(f"Current lr: {lr_now}")

    if no_improve_epochs >= patience:
        print("Early stopping triggered.")
        break


Eval: 100%|██████████| 13/13 [00:12<00:00,  1.02it/s]


Epoch 1/50 | Train Loss: 0.9607 | Val Loss: 0.9527 | IoU: 0.0235 | F1: 0.0426 | Precision: 0.0237 | Recall: 0.8766 | Accuracy: 0.4044
New best model saved (F1: 0.0426)
Current lr: 0.0001


Eval: 100%|██████████| 13/13 [00:12<00:00,  1.02it/s]


Epoch 2/50 | Train Loss: 0.9511 | Val Loss: 0.9456 | IoU: 0.0317 | F1: 0.0565 | Precision: 0.0337 | Recall: 0.8127 | Accuracy: 0.6308
New best model saved (F1: 0.0565)
Current lr: 0.0001


Eval: 100%|██████████| 13/13 [00:12<00:00,  1.02it/s]


...

Epoch 43/50 | Train Loss: 0.4484 | Val Loss: 0.7725 | IoU: 0.1956 | F1: 0.3045 | Precision: 0.4267 | Recall: 0.2639 | Accuracy: 0.9846
No improvement in F1 for 8 epoch(s)
Current lr: 1.0000000000000002e-07


Eval: 100%|██████████| 13/13 [00:12<00:00,  1.05it/s]


Epoch 44/50 | Train Loss: 0.4483 | Val Loss: 0.7805 | IoU: 0.1903 | F1: 0.2973 | Precision: 0.4326 | Recall: 0.2520 | Accuracy: 0.9847
No improvement in F1 for 9 epoch(s)
Current lr: 1.0000000000000002e-07


Eval: 100%|██████████| 13/13 [00:12<00:00,  1.05it/s]

Epoch 45/50 | Train Loss: 0.4482 | Val Loss: 0.7668 | IoU: 0.2011 | F1: 0.3097 | Precision: 0.4185 | Recall: 0.2741 | Accuracy: 0.9844
No improvement in F1 for 10 epoch(s)
Current lr: 1.0000000000000004e-08
Early stopping triggered.


In [ ]:
# Final evaluation with visualisations
print("Generating visualizations for best model")
model.load_state_dict(torch.load(os.path.join(output_dir, "best_model.pth")))
_, final_metrics = evaluate(model, test_loader, criterion, save_preds=True)
print("Final Test Metrics:")
for k, v in final_metrics.items():
    print(f"  {k.capitalize()}: {v:.4f}")
print(f"Predictions & visualizations saved to: {output_dir}")

Generating visualizations for best model


Eval: 100%|██████████| 14/14 [00:18<00:00,  1.30s/it]

Final Test Metrics:
  Accuracy: 0.9891
  Precision: 0.4709
  Recall: 0.2234
  F1_score: 0.2789
  Jaccard: 0.1720
Predictions & visualizations saved to: ./predictions
